In [8]:
import pandas as pd
import time
import datetime as dt
from binance.spot import Spot as SpotClient

In [9]:
client = SpotClient()  # pas besoin de clés pour les endpoints publics

def fetch_klines_spot_connector(symbol="BTCUSDT", interval="15m", start_ms=None, end_ms=None, limit=1000):                            
    """
    Fetches historical candlestick (kline) data from Binance Spot API and returns it as a pandas DataFrame.

    Args:
        symbol (str): Trading pair symbol (e.g., "BTCUSDT").
        interval (str): Kline interval (e.g., "15m", "1h").
        start_ms (int, optional): Start time in milliseconds since epoch.
        end_ms (int, optional): End time in milliseconds since epoch.
        limit (int): Maximum number of klines to fetch (default: 1000).

    Returns:
        pd.DataFrame: DataFrame containing kline data with columns:
            ['openTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime',
             'quoteAssetVolume', 'numberOfTrades', 'takerBuyBase', 'takerBuyQuote', 'ignore']
            Numeric columns are converted to float, and time columns to datetime (UTC).
    """
    
    rows = client.klines(symbol, interval, startTime=start_ms, endTime=end_ms, limit=limit)
    cols = ["openTime","open","high","low","close","volume","closeTime",
            "quoteAssetVolume","numberOfTrades","takerBuyBase","takerBuyQuote","ignore"]
    df = pd.DataFrame(rows, columns=cols)
    for c in ["open","high","low","close","volume","quoteAssetVolume","takerBuyBase","takerBuyQuote"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df["openTime"]  = pd.to_datetime(df["openTime"], unit="ms", utc=True)
    df["closeTime"] = pd.to_datetime(df["closeTime"], unit="ms", utc=True)
    return df

In [10]:
def fetch_klines_full(symbol="BTCUSDT", interval="1h",
                      start="2021-01-01 00:00:00", end="2021-02-01 00:00:00"):
    """
    Récupère tout l'historique entre start et end (UTC) en bouclant par tranches de 1000.
    """
    start_dt = dt.datetime.strptime(start, "%Y-%m-%d %H:%M:%S").replace(tzinfo=dt.timezone.utc)
    end_dt   = dt.datetime.strptime(end, "%Y-%m-%d %H:%M:%S").replace(tzinfo=dt.timezone.utc)

    start_ms = int(start_dt.timestamp() * 1000)
    end_ms   = int(end_dt.timestamp() * 1000)

    all_dfs = []
    cur_start = start_ms

    while cur_start < end_ms:
        df = fetch_klines_spot_connector(symbol, interval,
                                         start_ms=cur_start, end_ms=end_ms, limit=1000)
        if df.empty:
            break
        all_dfs.append(df)
        # avancer d’1 ms après le dernier close récupéré
        last_close = int(df["closeTime"].iloc[-1].timestamp() * 1000)
        cur_start = last_close + 1
        time.sleep(0.2)  # pour éviter le rate limit

    if all_dfs:
        return pd.concat(all_dfs, ignore_index=True)
    else:
        return pd.DataFrame()

In [16]:
# Exemple d’utilisation :
df = fetch_klines_full("BTCUSDT", "15m", "2021-01-01 00:00:00", "2025-09-01 00:00:00")

In [17]:
df.tail()

,openTime,open,high,low,close,volume,closeTime,quoteAssetVolume,numberOfTrades,takerBuyBase,takerBuyQuote,ignore
163510,2025-08-31 23:00:00+00:00,108900.45,108917.36,108622.65,108684.74,110.87622,2025-08-31 23:14:59.999000+00:00,1.205577e+07,19024,25.25046,2.744772e+06,0
163511,2025-08-31 23:15:00+00:00,108684.74,108684.74,108355.00,108387.96,162.49957,2025-08-31 23:29:59.999000+00:00,1.762948e+07,32177,61.78856,6.703993e+06,0
163512,2025-08-31 23:30:00+00:00,108387.96,108416.35,108076.93,108258.00,258.06379,2025-08-31 23:44:59.999000+00:00,2.792994e+07,42124,140.22900,1.517864e+07,0
163513,2025-08-31 23:45:00+00:00,108257.99,108356.05,108167.34,108246.35,166.04275,2025-08-31 23:59:59.999000+00:00,1.797487e+07,28333,86.02752,9.313063e+06,0
163514,2025-09-01 00:00:00+00:00,108246.36,108332.35,107970.65,107970.66,422.51571,2025-09-01 00:14:59.999000+00:00,4.566634e+07,47851,126.58885,1.368081e+07,0


In [18]:
df.to_csv("../data/BTC_15min",index=False)